# pyTigerGraph 101

## Connecting to the Database with pyTigerGraph

In [ ]:
import pyTigerGraph as tg

Some additional imports to help the demo run smoothly

In [ ]:
import json
import pandas as pd

In [ ]:
from pyTigerGraph import TigerGraphConnection

# Read in DB configs
with open('../config.json', "r") as config_file:
    config = json.load(config_file)
    
conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"]
)

## Load Example Dataset

In [ ]:
from pyTigerGraph.datasets import Datasets

dataset = Datasets("movie")

conn.ingestDataset(dataset, getToken=True)

# The Basics


This first set of commands will help us get familiar with the schema of our graph and explore what we're working with.

## Getting the Schema
Now let's take a peek at our schema to see what our graph looks like.

In [ ]:
results = conn.getSchema()
print(json.dumps(results, indent=2))

### Reading the Output
Woah, that's a lot of text! Let's break it down and see what we're really looking at. 
```
"GraphName": "movie",
```
That one is easy, that's our graph name.

#### Verticies
```
"VertexTypes": [
    {
      "Config": {
        "STATS": "OUTDEGREE_BY_EDGETYPE"
      },
      "IsLocal": true,
      "Attributes": [
        {
          "AttributeType": {
            "Name": "STRING"
          },
          "AttributeName": "name"
        },
        {
          "AttributeType": {
            "Name": "STRING"
          },
          "AttributeName": "known_label"
        },
        {
          "AttributeType": {
            "Name": "STRING"
          },
          "AttributeName": "predicted_label"
        }
      ],
      "PrimaryId": {
        "AttributeType": {
          "Name": "STRING"
        },
        "AttributeName": "name"
      },
      "Name": "Person"
    },
```
This is our first vertex type. We can see via the **Name** property at the bottom that this is our **Person** vertex.
Digging in a bit more, we can see that our **Person** vertex **has three attributes** (name, known_label, and predicted_label). 


Using this, can you figure out what our other vertex is and what its attributes are?


#### Edges
Next are our edge types
```
"EdgeTypes": [
    {
      "IsDirected": true,
      "ToVertexTypeName": "Movie",
      "Config": {
        "REVERSE_EDGE": "reverse_Likes"
      },
      "IsLocal": true,
      "Attributes": [
        {
          "AttributeType": {
            "Name": "FLOAT"
          },
          "AttributeName": "weight"
        }
      ],
      "FromVertexTypeName": "Person",
      "Name": "Likes"
    },
```
This follows the same idea as our verticies. 
We have an edge **Likes** with a reverse edge **reverse_Likes**. This edges goes **FromVertexTypeName**: **Person** to **ToVertexTypeName**: **Movie**. Additionally, this edge contains one attribute, **weight**.

That's the gist of our graph. It's simple, but it's a great starting point. We have users and movies with an edge connecting them containing the rating that the user gave the movie and the time it was rated at.

## Getting the Schema II
While the `getSchema()` command is great for giving you an overview of your whole graph, sometimes you want... less.

### Getting Types

In [ ]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")

results = conn.getEdgeTypes()
print(f"Edges: {results}")

`getVertexTypes()` and `getEdgeTypes()` will both return a list of strings naming each type of requested graph component.

Furthermore, you can use the output of those commands to individually inspect each vertex or edge type.

### Getting Info by Type

In [ ]:
results = conn.getVertexType("Person")
print(f"Person Vertex Attributes:\n {json.dumps(results, indent=2)}")
print("---------------")

results = conn.getEdgeType("Likes")
print(f"Rate Edge Attributes:\n {json.dumps(results, indent=2)}")

### Detailed Edge Info
And going even further, you can deep dive into each edge endpoint like so:

In [ ]:
sourceVertex = conn.getEdgeSourceVertexType("Likes")

targetVertex = conn.getEdgeTargetVertexType("Likes")

directed = conn.isDirected("Likes")
directedText = "is" if directed==True else "is not"

reverseEdge = conn.getReverseEdge("Likes")

print(f"Edge 'Likes' '{directedText} directed' with a source vertex type '{sourceVertex}' and a target vertex type '{targetVertex}'")
if directed:
  print(f"The reverse edge is '{reverseEdge}'")

## Looking at Data
Next let's see what we have for data in our graph.

These next commands will count the total number of verticies and edges we currently have data loaded for. We'll define these as a function called `getLoadedStats` so we can use it later on.

In [ ]:
def getLoadedStats(limit=5):
  numPeople = conn.getVertexCount("Person")
  numMovies = conn.getVertexCount("Movie")
  numEdges = conn.getEdgeCount("Likes")

  people = conn.getVertices("Person", limit=limit)
  movies = conn.getVertices("Movie", limit=limit)
  # edges = conn.getEdgesByType("rate", limit=limit)

  print(f"There are currently {numPeople} people, {numMovies} movies, and {numEdges} edges connecting them")
  print(f"Our people are: {json.dumps(people, indent=2)}")
  print(f"Our movies are: {json.dumps(movies, indent=2)}")
  # print(f"Our edges are: {json.dumps(edges, indent=2)}")

getLoadedStats()

# Adding Data
Well, a graph without data is no fun, so let's get some data in here. 

I'll start off by showing you how to add data one datapoint at a time, then we'll move into bulk data loads via loading jobs.

## Per-value Entry
This is how to load a single vertex/edge at a time. This is useful for troubleshooting data and understanding your data format.

Let's add a **person**, a **movie**, and a **review** to our graph.

### Adding a Person
Our person will be our simplest datapoint to add because it doesn't have any attributes, just our **PrimaryId**. 

The inputs for our `upsertVertex` function are:
- vertexType - "person" for this example
- vertexId - an ID unique to our vertex, in this case the person's name
- attributes - a json structure of the vertex's attributes

It is important to realize here that this is an **upsert** meaning **update/insert**. If the vertexId is **not present** in the graph, a new vertex will be created with the included data. If that vertexId is **already present** in the graph, then the exiting vertex's attributes will be updated to match the current request. 

In [ ]:
results = conn.upsertVertex("Person", "Dan", {})
print(results)

A response of 1 means that our data has successfully been loaded!

Let's check with our `getLoadedStats` function.

In [ ]:
getLoadedStats()

Sweet!

Time to add a movie.

### Adding a Movie
If you remember from our schema, our **movie** vertex contains some attributes that we'll want to enter. These are the movie's **title** and **genres**. 

I'm a fan of brain-dead action movies, so let's get a movie loaded up.

In [ ]:
attributes = {"name": "Die Hard 4: Live Free or Die Hard"}
results = conn.upsertVertex("Movie", "1", attributes)
print(results)

Time for the rewarding bit again

In [ ]:
getLoadedStats()

Yay! More data!

Time to make an edge!

### Add a Like
Adding an edge isn't all that different from adding a vertex. Main difference is that you need to point it at two existing vertices.

Here's our inputs for `upsertEdge`:
- sourceVertexType - "Person" for us
- sourceVertexId - **PrimaryId** of our source Person
- edgeType - "rate" for this example
- targetVertexType - "Movie" here
- targetVertexId - **PrimaryId** of the target Movie
- attributes - json structured attributes and values

Looks like a lot, but it's pretty simple in practice. Again, remember that this is an **upsert**.

In [ ]:
attributes = {"weight": 8.6}
results = conn.upsertEdge("Person", "Dan", "Likes", "Movie", "1", attributes)
print(results)

You know what step is next

In [ ]:
getLoadedStats()

Boom! That's it, we're done here.

Just kidding, one connection is good and all, but it really doesn't show off the power of graph. Also we're only through 1/4 of the functionality that pyTigerGraph offers, so let's pick things up a bit.

## Load Multiple Vertices/Edges at Once
Obviously adding data points one by one is a pain and really only useful for extremely small amounts of data. This next function will allow you to load multiple points of data all at once.

### Formatting the Data
You'll need your data in a similar format to what was used in the above commands for these bulk commands. If your data is in a .csv or json format, we have loaders specifically for those types of files and you'll want to use those. For the sake of completeness, I'm going to show you this method in the event that your data is being generated from a script, or is already in this format.

In [ ]:
# Vertex format [(PrimaryId, {attributes})]
people = [
          ("Ben", {}),
          ("Nick", {}),
          ("Leena", {})
          ]
movies = [
          (2, {"name": "Inception"}),
          (3, {"name": "Her"}),
          (4, {"name": "Ferris Bueller's Day Off"})
          ]
# Edge format [(SourcePrimaryId, TargetPrimaryId, {attributes})]
ratings = [
           ("Ben", 2, {"weight": 7.3}),
           ("Nick", 4, {"weight": 9.2}),
           ("Nick", 3, {"weight": 8.7}),
]

You can see that in the above data, the **vertexType** is not specified for the vertices and the **edgeType**, **SourceVertexType**, and **TargetVertexType** are not specified for the edges.

This is because these will be specified when running the upsert command. The big thing to note here is that **only ONE type of vertex or edge can be loaded per upsert**. This means that I can't store both my **movies** and my **people** in the same array and expect the upsert to be able to work it out.

### Loading the Data

In [ ]:
results = conn.upsertVertices("Person", people)
print(results)
results = conn.upsertVertices("Movie", movies)
print(results)

results = conn.upsertEdges("Person", "Likes", "Movie", ratings)
print(results)

Here you'll see our output number signifies the number of each vertices/edges loaded successfully.

And if we run our good friend again...

In [ ]:
getLoadedStats()

Look at all that data!

Let's look at a couple more ways we can explore it.

# Looking at Data II

## Stats
One helpful way to look at our data is to collect some overall stats for each vertex or edge type.

Let's take a look at how to do that with the `getVertexStats` and `getEdgeStats` functions.

In [ ]:
results = conn.getVertexStats("Person")
print(json.dumps(results, indent=2))

results = conn.getVertexStats("Movie")
print(json.dumps(results, indent=2))

results = conn.getEdgeStats("Likes")
print(json.dumps(results, indent=2))
print("----------")

results = conn.getVertexStats("*", skipNA=True)
print(json.dumps(results, indent=2))

As we can see from the results, we only received stats from our edge. That might not seem right at first, but this is actually the expected behavior.

If we look at what was returned from our edge stats, we can see we get a **MAX**, **MIN**, and **AVG**. This makes sense for our edge because our attribute types are **float** for the **weight**.

Finally, in the last command, we used "*" as the vertex type. This is the **wildcard** operator and signifies all vertex types in this instance. Additionally, the **skipNA** flag will tell pytigerGraph not to return results for vertices which stats are not applicable. You can see that that command just returns an empty json without the vertex type names.

## Specific Vertices
We can use a vertex's **PrimaryId** to return all information that we have on that vertex. Let's try it now.

In [ ]:
results = conn.getVerticesById("Person", "Dan")
print(json.dumps(results, indent=2))
print("----------")

results = conn.getVerticesById("Movie", ["2","4"])
print(json.dumps(results, indent=2))

Note that we still need to provide the **vertexType** due to **PrimaryId** only being unique within each **vertexType**.

Also note that we can provide a list of **PrimaryId**s in order to return multiple vertices at once.

## Edges
In addition to being able to inspect vertices, we can do the same with edges.

In [ ]:
results = conn.getEdgeCountFrom("Person", "Nick")
print("Number of edges connected to person 'Nick'")
print(results)
print("--------------")
# Can also specify edge type and/or target vertex type and target vertex id
results = conn.getEdgeCountFrom("Person", "Nick", edgeType="Likes")
print("Number of 'rate' edges connected to person 'Nick'")
print(results)
print("--------------")
# Get all edges connected to a starting vertex
results = conn.getEdges("Person", "Nick")
print("All edges connected to person 'Nick'")
print(json.dumps(results, indent=2))

# Deleting Data
Now that we know how to create data and view it in our graph, it's only logical that we learn how to delete it. 

There are three main ways to delete data via pyTigerGraph:
- Per vertex
- Multiple vertices/edges following a condition
- All vertices/edges

Before we start deleting things, let's make sure we remember what the data in our graph currently looks like.

In [ ]:
getLoadedStats()

## Single Vertex

Our "Leena" person hasn't rated any movies, so let's delete them from the graph.

In [ ]:
results = conn.delVerticesById("Person", "Leena")
print(results)

Our output will tell us the number of verteices that we deleted and we should see 1 representing Leena.

### Permenant Flag
For each of our delete commands, we have the option of the `permanent` flag. While deleting with a false `permenant` flag (the default) **will** delete the data from your graph, you can still reinsert that data back into the graph. The `permenant` flag makes it so that any vertex id's that are being deleted from the graph **cannot** be re-inserted unless the graph is dropped or the graph store is cleared.

## Multiple Vertices/edges
We can use **WHERE**, **LIMIT**, and **SORT** conditions to filter vertices and edges that we want to delete. I'll be showing off **WHERE** here as we don't have enough data to really make use of the others. 

Documentation on each of the supported clauses can be found here:
- [WHERE](https://docs.tigergraph.com/dev/restpp-api/built-in-endpoints#filter) 
- [LIMIT](https://docs.tigergraph.com/dev/restpp-api/built-in-endpoints#limit)
- [SORT](https://docs.tigergraph.com/dev/restpp-api/built-in-endpoints#sort)

Let's say that we don't want to retain any ratings that are less than a '9.0'. Whe can use a **WHERE** clause to filter and delete any edges where the rating is less than '9.0'

In [ ]:
# Stats before deleting ratings below 9.0
results = conn.getEdgeStats("Likes")
edgeCnt = conn.getEdgeCount(edgeType="Likes")
print("Before deleting ratings less than '9.0'")
print(f"{edgeCnt} 'Likes' edges")
print(json.dumps(results, indent=2))
print("-------------")

results = conn.delEdges("Person", "Nick", edgeType="Likes", where="weight < 9.0")
print(results)
print("-------------")

results = conn.getEdgeStats("Likes")
edgeCnt = conn.getEdgeCount(edgeType="Likes")
print("After deleting ratings less than '9.0' that Nick made")
print(f"{edgeCnt} 'Likes' edges")
print(json.dumps(results, indent=2))

That didn't quite do what we wanted. As you can see, we still have edges with a rating below 9.0. This is because we could only check edges coming from one vertex at a time.

Let's take a look at how we can use some of the commands that we learned earlier to take a more programatic approach to this.

In [ ]:
# Get all 'rate' edges
edges = conn.getEdgesByType("Likes")

# Delete any edges with a rating less than 9.0
for edge in edges:
  if edge["attributes"]["weight"] < 9.0:
    rating = edge["attributes"]["weight"]
    fromPerson = edge["from_id"]
    deleted = conn.delEdges("Person", edge["from_id"])
    print(f"Deleting a rating of {rating} from {fromPerson}")
    
print("-------------")
results = conn.getEdgeStats("Likes")
edgeCnt = conn.getEdgeCount(edgeType="Likes")
print("After deleting ratings less than '9.0'")
print(f"{edgeCnt} 'rate' edges")
print(json.dumps(results, indent=2))

## Deleting all Vertices
Here's the big one, time to delete all our vertices in the graph. We'll load in more data so we can continue later, but it's time to bring it all down now.

In [ ]:
results = conn.delVertices("Person")
print(f"deleted {results} people")
results = conn.delVertices("Movie")
print(f"deleted {results} movies")

In [ ]:
getLoadedStats()

# Bulk Data Upsertion
We went over how to load data bit by bit, but if you wanted a graph database that could only handle small amounts of data, you'd be using Neo4j. Let's look at the two methods you can use for bulk data loading.

## Loading Via JSON with `UpsertData`
This method is useful if you have data that has been exported in a JSON format from another graph, or if you have a JSON formatted file that you want to upload.

The format of this file does need to follow the format specified [in the documentation](https://docs.tigergraph.com/dev/restpp-api/built-in-endpoints#upsert-data-to-graph).

I'll also list out the basic structure here
```
{
  "vertices": {
     "<vertex_type>": {
        "<vertex_id>": {
           "<attribute>": {
              "value": <value>,
              "op": <opcode>
           }
        }
    }
  },
  "edges": {
     "<source_vertex_type>": {
        "<source_vertex_id>": {
           "<edge_type>": {
              "<target_vertex_type>": {
                 "<target_vertex_id>": {
                    "<attribute>": {
                       "value": <value>,
                       "op": <opcode>
                    }
                }
              }
          }
        }
    }
  }
}
```
Let's give it a shot with a JSON of our own.

In [ ]:
data = {
    "vertices": {
        "Person": {
            "Dan": {
            },
            "Ben": {  
            }
          },
          "Movie": {
              "1": {
                  "name": {
                      "value": "Up"
                  }
              },
              "2": {
                  "name": {
                      "value": "Redline"
                  },
              }
          }
        },
        "edges": {
            "Person": {
                "Dan": {
                    "Likes": {
                        "Movie": {
                            "1": {
                                "weight": {
                                    "value": 8.7
                                },
                            },
                            "2": {
                                "weight": {
                                    "value": 9.1
                                },
                            }
                        }
                    }
                },
                "Ben": {
                    "Likes": {
                        "Movie": {
                            "1": {
                                "weight": {
                                    "value": 8.2
                                },
                            },
                            "2": {
                                "weight": {
                                    "value": 7.6
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    
results = conn.upsertData(data)
print(results)

In [ ]:
getLoadedStats()

So that's not exactly the huge quantity of data that I promised, but it gives you the idea of how you could load a much larger JSON object or file. 

## Loading Jobs (your best bet)
While JSON and individual loading are powerful, loading jobs really help TigerGraph shine. These jobs run off .csv files loaded onto your TigerGraph server and can rapidly load absolutely abusrd amounts of data onto your TigerGraph solution.

As part of our demo solution, we already have a couple loading jobs created. You can view these through the **Load Data** tab in GraphStudio, or by using the `ls` command through a GSQL interface.

Luckily pyTigerGraph can give us that GSQL interface. Let's have a look!

In [ ]:
print(conn.gsql('ls'))

### GSQL

In [ ]:
results = conn.gsql('''
USE GRAPH movie
SHOW JOB *''')
print(results)

These are our **loading jobs**. **Loading Jobs** are mappings between data files and vertices/edges. They are how you take data in a .csv or .json and assign it to vertices and create edges. These **loading jobs** also correspond to our data mapping in the **Map Data to Graph** tab in GraphStudio.

This was automatically generated and ran when we imported the `movie` dataset at the beginning of the notebook.

In [ ]:
getLoadedStats()

# Queries
Now that all of our data is loaded, let's take a look at how to run some queries on it. 

We're going to need to start with creating a query, then we'll cover running them.

## Creating a new Query
Once again, we can use the handy GSQL functionality built into pyTigerGraph to create a query on our TigerGraph solution.

Query creation follows the rules outlined [here](https://docs.tigergraph.com/dev/gsql-ref/querying/query-operations#create-query) in the documentation.

Here's a simple sample query.

In [ ]:
results = conn.gsql('''
USE GRAPH movie
CREATE QUERY testQuery() FOR GRAPH movie { PRINT "testQuery works!"; }''')
print(results)

## Running a Query in Interpreted Mode
That test query that we just added has been **saved** on our TigerGraph solution. It's important to note that this qurey is **saved** not **installed**. [This link](https://docs.tigergraph.com/dev/gsql-ref/querying/query-operations#interpret-query) will help you understand the difference and what that means in terms of functionality and performance.

In [ ]:
results = conn.gsql('''
USE GRAPH movie
INTERPRET QUERY testQuery ()''')
print(results)

While that command **saved** the 'testQuery' on our TigerGraph solution, we don't need to have a saved query in order to run it in interpreted mode.

We can use the pyTigerGraph function `runInterpretedQuery` to run a one-off query in interpreted mode without saving or installing it. 

In [ ]:
results = conn.runInterpretedQuery('''INTERPRET QUERY testQuery2() FOR GRAPH movie { PRINT "testQuery2 works!"; }''')
print(results)

Note here that we're using **INTERPRET QUERY** instead of the **RUN QUERY** that was used when saving the query.

## Installing a Query
Let's use GSQL again to **install** a more complicated query on our Solution.

We'll install a query that uses cosine similarity to try to recommend movies to one person based on what movies people with similar movie interests to them have rated highly.

I'll go over the query in more detail when we go to run it.

In [ ]:
result = conn.gsql('''USE GRAPH movie
CREATE QUERY RecommendMovies(
 VERTEX<Person> p, int k1, int k2
) FOR GRAPH movie {
 OrAccum @rated;
 SumAccum<double> @ratingByP;
 SumAccum<double> @lengthASqr, @lengthBSqr, @dotProductAB;
 SumAccum<double> @cosineSimilarity;
 AvgAccum @recommendScore;

 PSet = { p };
	
 PRatedMovies =
    SELECT m
    FROM PSet -(Likes:r)-> Movie:m
    ACCUM m.@rated = true, m.@ratingByP = r.weight;

 PeopleRatedSameMovies =
    SELECT tgt
    FROM PRatedMovies:m -(:r)-> Person:tgt
    WHERE tgt != p
    ACCUM tgt.@lengthASqr += m.@ratingByP * m.@ratingByP,
          tgt.@lengthBSqr += r.weight * r.weight,
          tgt.@dotProductAB += m.@ratingByP * r.weight
    POST-ACCUM
          tgt.@cosineSimilarity =
            tgt.@dotProductAB / (sqrt(tgt.@lengthASqr) * sqrt(tgt.@lengthBSqr))
    ORDER BY tgt.@cosineSimilarity DESC
    LIMIT k1;

  RecommendedMovies =
    SELECT m
    FROM PeopleRatedSameMovies -(Likes:r)-> Movie:m
    WHERE m.@rated == false
    ACCUM m.@recommendScore += r.weight
    ORDER BY m.@recommendScore DESC
    LIMIT k2;

  PRINT RecommendedMovies;
}''')
print(result)

Our query is saved, but not yet installed.

Let's do that now. This **will** take 1-3 minutes, so sit tight.

In [ ]:
print(conn.gsql('''
USE GRAPH movie
INSTALL QUERY RecommendMovies'''))

## Running a Query
Now we can run our installed query. Let's take a look at the input parameters for our query and what they do.

Our RecommenedMovies query has 3 inputs:
- p - person who we're looking to recommend movies to
- k1 - number of people who rated the same movies as p to consider
- k2 - number of movies to recommend

In [ ]:
parameters = {
    "p": "Dan",
    "k1": 50,
    "k2": 10
}

results = conn.runInstalledQuery("RecommendMovies", params=parameters)
parsed = conn.parseQueryOutput(results)
print(json.dumps(parsed, indent=2))
# print(json.dumps(results, indent=2))

Our query result will output 'k2'(10) movies that our input person 'p'("215") has not seen and are similar to movies that they have rated highly.

Go ahead and play around with those inputs to see what kind of results you can get.

Additionally, we used the `parseQueryOutput` to separate out the vertices and edges in our result and present it in a more pleasant fashion.

# Path Finding
We can traverse the edges of our graph to find unique connections between points of data.

For this graph, we only have people and movies, but we can use that to find paths between people based on movies that they've rated. Let's take a look.

## Shortest Path
This function finds the shortest path between two or more vertices.

In [ ]:
results = conn.shortestPath([("Person", "Dan")], [("Person", "Ben")])
print(json.dumps(results, indent=2))

You can see that **person 215** and **person 777** are connected because they have both rated **American Splendor**.

Another thing to note here is the inputs for the function. Our inputs can be either
- vertex sets - JSON vertex set
- list of vertex, type tuples - `[(VERTEX_TYPE, VERTEX_ID)]`

## All Paths
Get all paths between the selected vertices. In the case of our graph, this will return all common movies that two people have rated. It will also return a movie that person A has rated that person X has rated that person B has also rated. Needless to say, there are a lot of possible path combinations, so an upper limit to the number of edge traversals is required for this query.

In [ ]:
results = conn.allPaths([("Person", "Dan")], [("Person", "Ben")], 4)
print(json.dumps(results, indent=2))

# Pandas
pyTigerGraph includes many functions for integrating with Pandas Dataframes.

## Retrieve Data as Dataframe
These functions allow you to query vertex and edge data from the graph and return a Pandas dataframe.

### Vertex Information

In [ ]:
# Get multiple vertices of a type
df1 = conn.getVertexDataFrame("Movie")
print("Movies Dataframe")
print(df1)
print("---------------------")

# Specifically pick one or more vertex by id
df2 = conn.getVertexDataFrameById("Movie", ["1"])
print("Specific Movies Dataframe")
print(df2)
print("---------------------")

# Convert a vertex set to a dataframe
result = conn.getVerticesById("Person", "Dan")
print("JSON Vertex Set")
print(json.dumps(result, indent=2))
print("---------------------")

df3 = conn.vertexSetToDataFrame(result)
print("Above Vertex Set converted to Pandas Dataframe")
print(df3)

### Edge Information

In [ ]:
# Get all edges from selected starting vertex
df4 = conn.getEdgesDataframe("Person", "Dan", limit=3)
print("Edges Dataframe")
print(df4)
print("---------------------")

result = conn.getEdges("Person", "Dan", limit=3)
print("JSON Edge Set")
print(json.dumps(result, indent=2))
print("---------------------")

df5 = conn.edgeSetToDataFrame(result)
print("Converted Edge Set as Dataframe")
print(df5)

# Other
Useful functions of pyTigerGraph that may be used less frequently, or just other useful blocks of code when interacting with pyTigerGraph.

In [ ]:
def getLoadedStats(limit=5):
  numPeople = conn.getVertexCount("person")
  numMovies = conn.getVertexCount("movie")
  numEdges = conn.getEdgeCount("rate")

  people = conn.getVertices("person", limit=limit)
  movies = conn.getVertices("movie", limit=limit)

  print(f"There are currently {numPeople} people, {numMovies} movies, and {numEdges} edges connecting them")
  print(f"Our people are: {json.dumps(people, indent=2)}")
  print(f"Our movies are: {json.dumps(movies, indent=2)}")
  print(f"Our edges are: {json.dumps(edges, indent=2)}")

print("function 'getLoadedStats()' created")

## Secret Management
When you **first** connect pyTigerGraph to a Solution, you will need to create the secret elsewhere (because if pyTigerGraph needs a secret to connect, it can't connect to create the first secret).

You **can** however create additional secrets once you're connected and even view all the secrets that your user account has access to.

In [ ]:
secret = conn.createSecret(alias="testSecret2")

print(secret)
print("---------------")
print(conn.getSecrets())

## Token Management
These functions allow you to manage your connection tokens.

Tokens can be given a lifespan with `lifetime=` where the value of lifetime is the token's lifespan in seconds.

In [ ]:
# Will give you a new token
print(conn.refreshToken(secret, lifetime=2592000)) # 2,592,000 is 30 days worth of seconds


# Deletes an existing token
# print(conn.deleteToken(secret))

## Solution Info
You can view all sorts of stats and information about your TigerGraph solution as a whole.

### echo
Check to see if Solution is online and responds with "Hello GSQL"

In [ ]:
print(conn.echo())

### getEndpoints
A lot of TigerGraph's functionality is exposed through REST endpoints. To see the endpoints available to you on your Solution, use `getEndpoints()`. 

Further documentation on these endpoints can be found [here](https://docs.tigergraph.com/dev/restpp-api/built-in-endpoints).

In [ ]:
print(json.dumps(conn.getEndpoints(), indent=2))

### getInstalledQueries
This command is just and easier way of doing `GSQL SHOW QUERIES *`.

In [ ]:
print(json.dumps(conn.getInstalledQueries(),indent=2))

### getStatistics
Returns statistics of query execution over the last timespan in seconds.

Inputs:
- seconds - duration of statistics to collect (T-seconds from execution time)
- segments - segments of latency distribution (default is 10 max is 100, min is 1)

In [ ]:
print(json.dumps(conn.getStatistics(seconds=8640000), indent=2))

### getVer and getVersion
These commands return version information about the various services running on your TigerGraph Solution.

**getVersion** will return the version of all components while **getVer** takes an input of a component name and will only return the version for that component.

In [ ]:
print(json.dumps(conn.getVersion(), indent=2))
print("-----------------")
print(json.dumps(conn.getVer("gle"), indent=2))

### getUDT and getUDTs
UDT's are User Defined Types. You can read about them [here](https://docs.tigergraph.com/dev/gsql-ref/querying/data-types#tuple).

There aren't any in our current graph, so both commands should return empty.

- getUDTs - returns all UDTs
- getUDT - only returns UDT of the defined input type 

In [ ]:
print(conn.getUDTs())
print(conn.getUDT(udtName=""))

## Cleanup
Since we altered the stock dataset graph, we are going to drop it so it can be re-loaded in other notebooks.

In [ ]:
conn.gsql("USE GRAPH {}\nDROP JOB ALL\nDROP Query ALL\nDROP GRAPH {}".format(
                 conn.graphname, conn.graphname
            ))